In [46]:

import pandas as pd
from bs4 import BeautifulSoup
import time
from datetime import datetime

!pip install selenium
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [24]:
df = pd.read_csv('../data/raw/coinmarketcap_cryptocurrency_listings_latest.csv')
df.head()

,id,name,symbol,slug,num_market_pairs,date_added,tags,max_supply,circulating_supply,total_supply,infinite_supply,platform,cmc_rank,self_reported_circulating_supply,self_reported_market_cap,tvl_ratio,last_updated,quote
0,1,Bitcoin,BTC,bitcoin,10943,2010-07-13T00:00:00.000Z,"['mineable', 'pow', 'sha-256', 'store-of-value...",21000000.0,1.966502e+07,1.966502e+07,False,NaN,1,NaN,NaN,NaN,2024-03-26T18:25:00.000Z,"{'USD': {'price': 70215.2092211454, 'volume_24..."
1,1027,Ethereum,ETH,ethereum,8652,2015-08-07T00:00:00.000Z,"['pos', 'smart-contracts', 'ethereum-ecosystem...",NaN,1.200740e+08,1.200740e+08,True,NaN,2,NaN,NaN,NaN,2024-03-26T18:24:00.000Z,"{'USD': {'price': 3590.2311684461333, 'volume_..."
2,825,Tether USDt,USDT,tether,80280,2015-02-25T00:00:00.000Z,"['payments', 'stablecoin', 'asset-backed-stabl...",NaN,1.042337e+11,1.077926e+11,True,"{'id': 1027, 'name': 'Ethereum', 'symbol': 'ET...",3,NaN,NaN,NaN,2024-03-26T18:24:00.000Z,"{'USD': {'price': 1.000137662205108, 'volume_2..."
3,1839,BNB,BNB,bnb,2124,2017-07-25T00:00:00.000Z,"['marketplace', 'centralized-exchange', 'payme...",NaN,1.495367e+08,1.495367e+08,False,NaN,4,NaN,NaN,NaN,2024-03-26T18:24:00.000Z,"{'USD': {'price': 579.0937507284682, 'volume_2..."
4,5426,Solana,SOL,solana,646,2020-04-10T00:00:00.000Z,"['pos', 'platform', 'solana-ecosystem', 'cms-h...",NaN,4.441814e+08,5.725844e+08,True,NaN,5,NaN,NaN,NaN,2024-03-26T18:24:00.000Z,"{'USD': {'price': 189.31256529881077, 'volume_..."


# Get historical data from 2020-2024 for each currency using url

In [25]:
base_url = 'https://coinmarketcap.com'

In [26]:
start_date = 20200101
end_date = 202403026

In [27]:
historical_url = f'{base_url}/currencies/bitcoin/historical-data/?start={start_date}&end={end_date}'
historical_url

'https://coinmarketcap.com/currencies/bitcoin/historical-data/?start=20200101&end=202403026'

In [28]:
driver = webdriver.Chrome()
driver.get(historical_url)

In [29]:
# wait for the history table to be generated
wait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '.history table')))


<selenium.webdriver.remote.webelement.WebElement (session="c981ff49a79be317ef31f846e9f46bb4", element="f.30D34A6FDBB7E93F7CA58A61C59DBA3E.d.D1F4BDFD12AAE0FB1D2F5728E21FF39A.e.22")>

In [30]:
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [31]:
history_div = soup.find('div', {'class': 'history'})
history_table = history_div.find('table')
history_table

<table class="sc-14cb040a-3 eGIvUX cmc-table"><colgroup><col style="width: 315px; min-width: auto;"/><col style="width: 120px; min-width: auto;"/><col style="width: 120px; min-width: auto;"/><col style="width: 120px; min-width: auto;"/><col style="width: 120px; min-width: auto;"/><col style="width: 315px; min-width: auto;"/><col style="width: 150px; min-width: auto;"/></colgroup><thead><tr><th class="stickyTop" style="text-align: start; top: 1px;">Date</th><th class="stickyTop" style="text-align: start; top: 1px;">Open*</th><th class="stickyTop" style="text-align: start; top: 1px;">High</th><th class="stickyTop" style="text-align: end; top: 1px;">Low</th><th class="stickyTop" style="text-align: end; top: 1px;">Close**</th><th class="stickyTop" style="text-align: end; top: 1px;">Volume</th><th class="stickyTop" style="text-align: end; top: 1px;">Market Cap</th></tr></thead><tbody><tr><td style="text-align: start;">Mar 26, 2024</td><td style="text-align: start;">$69,931.33</td><td style=

In [32]:
last_date_string = history_table.findAll('tr')[-1].find('td').text
last_date_string

'Jan 28, 2024'

In [33]:


last_date = datetime.strptime(last_date_string, '%b %d, %Y')
last_date

datetime.datetime(2024, 1, 28, 0, 0)

In [34]:
start_dateo = datetime.strptime(str(start_date), '%Y%m%d')
start_dateo

datetime.datetime(2020, 1, 1, 0, 0)

In [35]:
last_date <= start_dateo

False

In [36]:
def check_if_limit_reached(driver, start_date):
    try:
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        history_div = soup.find('div', {'class': 'history'})
        history_table = history_div.find('table')

        # Get the last date in the table
        last_date_string = history_table.findAll('tr')[-1].find('td').text
        last_date = datetime.strptime(last_date_string, '%b %d, %Y')
        from_date = datetime.strptime(str(start_date), '%Y%m%d')
        print("Last date in the table: ", last_date)
        print("start_date: ", from_date)
        
        if last_date <= from_date:
            return True
        return False
    
    except Exception as e:
        print('Exception check_if_limit_reached: ', e)
        return False

In [37]:
while True:
    try:
        # wait for load_more_button to be clickable
        load_more_button = wait.until(EC.presence_of_element_located((By.XPATH, "//button[text()='Load More']")))
        driver.execute_script("arguments[0].scrollIntoView();", load_more_button)
        driver.execute_script("arguments[0].click();", load_more_button)
        time.sleep(3)
        
        if check_if_limit_reached(driver, start_date):
            break
        
    except Exception as e:
        break

Last date in the table:  2023-11-29 00:00:00
start_date:  2020-01-01 00:00:00
Last date in the table:  2023-09-30 00:00:00
start_date:  2020-01-01 00:00:00
Last date in the table:  2023-07-31 00:00:00
start_date:  2020-01-01 00:00:00
Last date in the table:  2023-06-01 00:00:00
start_date:  2020-01-01 00:00:00
Last date in the table:  2023-04-02 00:00:00
start_date:  2020-01-01 00:00:00
Last date in the table:  2023-02-03 00:00:00
start_date:  2020-01-01 00:00:00
Last date in the table:  2022-12-04 00:00:00
start_date:  2020-01-01 00:00:00
Last date in the table:  2022-10-05 00:00:00
start_date:  2020-01-01 00:00:00
Last date in the table:  2022-08-06 00:00:00
start_date:  2020-01-01 00:00:00
Last date in the table:  2022-06-07 00:00:00
start_date:  2020-01-01 00:00:00
Last date in the table:  2022-04-08 00:00:00
start_date:  2020-01-01 00:00:00
Last date in the table:  2022-02-09 00:00:00
start_date:  2020-01-01 00:00:00
Last date in the table:  2021-12-10 00:00:00
start_date:  2020-0

In [38]:
soup = BeautifulSoup(driver.page_source, 'html.parser')
driver.quit()

In [39]:
history_div = soup.find('div', {'class': 'history'})
history_table = history_div.find('table')
print(history_table)

<table class="sc-14cb040a-3 eGIvUX cmc-table"><colgroup><col style="width: 315px; min-width: auto;"/><col style="width: 120px; min-width: auto;"/><col style="width: 120px; min-width: auto;"/><col style="width: 120px; min-width: auto;"/><col style="width: 120px; min-width: auto;"/><col style="width: 315px; min-width: auto;"/><col style="width: 150px; min-width: auto;"/></colgroup><thead><tr><th class="stickyTop" style="text-align: start; top: 1px;">Date</th><th class="stickyTop" style="text-align: start; top: 1px;">Open*</th><th class="stickyTop" style="text-align: start; top: 1px;">High</th><th class="stickyTop" style="text-align: end; top: 1px;">Low</th><th class="stickyTop" style="text-align: end; top: 1px;">Close**</th><th class="stickyTop" style="text-align: end; top: 1px;">Volume</th><th class="stickyTop" style="text-align: end; top: 1px;">Market Cap</th></tr></thead><tbody><tr><td style="text-align: start;">Mar 26, 2024</td><td style="text-align: start;">$69,931.33</td><td style=

In [40]:
history_df = pd.read_html(str(history_table))[0]
history_df.head()

,Date,Open*,High,Low,Close**,Volume,Market Cap
0,"Mar 26, 2024","$69,931.33","$71,535.74","$69,335.61","$69,987.84","$36,010,437,368","$1,376,403,313,500"
1,"Mar 25, 2024","$67,234.10","$71,162.59","$66,414.83","$69,958.81","$42,700,139,523","$1,375,150,091,237"
2,"Mar 24, 2024","$64,070.75","$67,622.76","$63,825.85","$67,234.17","$27,206,630,673","$1,322,058,484,489"
3,"Mar 23, 2024","$63,802.72","$65,976.40","$63,038.49","$64,062.20","$24,738,964,812","$1,259,788,729,091"
4,"Mar 22, 2024","$65,489.93","$66,623.75","$62,355.37","$63,778.76","$41,401,116,964","$1,254,457,600,698"


In [44]:
history_df.tail()

,Date,Open*,High,Low,Close**,Volume,Market Cap
1552,"Dec 26, 2019","$7,274.80","$7,388.30","$7,200.39","$7,238.97","$22,787,010,034","$131,200,030,100"
1553,"Dec 25, 2019","$7,325.76","$7,357.02","$7,220.99","$7,275.16","$21,559,505,149","$131,840,641,292"
1554,"Dec 24, 2019","$7,354.39","$7,535.72","$7,269.53","$7,322.53","$22,991,622,105","$132,687,583,295"
1555,"Dec 23, 2019","$7,508.90","$7,656.18","$7,326.19","$7,355.63","$27,831,788,041","$133,275,709,111"
1556,"Dec 22, 2019","$7,191.19","$7,518.03","$7,167.18","$7,511.59","$23,134,537,956","$136,088,022,049"


In [45]:
history_df['Date'].nunique()

1557

In [41]:
history_df.shape

(1557, 7)

In [42]:
history_df.to_csv('../data/raw/bitcoin_historical_data.csv', index=False)

In [43]:
for currency in df['slug'][:2]:
    print(currency)

bitcoin
ethereum
